In [0]:
import numpy as np

In [0]:
class Game2048(object):
    def __init__(self, n):
        self.n = n
        self.dims = 2
        self.n_news = ((n**self.dims)//2**4)*2
        self.shape = [n for i in range(self.dims)]
        self.actions = [(dim, 0) for dim in range(self.dims)] + [(dim, 1) for dim in range(self.dims)]
        self.reset()
    
    def reset(self):
        self.board = np.zeros(shape=self.shape)
        self.next_board = np.zeros(shape=self.shape)
        self.zero_mask = np.ones(shape=self.shape)
        self.score = 0
        self.last_move = False
        self.actions_taken = []
        self.add_numbers()
    
    def add_numbers(self):
        idx_gt_zero = np.argwhere(self.board)
        np.put(self.zero_mask, np.ravel_multi_index(idx_gt_zero.T, self.shape), 0, mode='raise')
        idx_zero = np.argwhere(self.zero_mask)
        np.put(self.zero_mask, np.ravel_multi_index(idx_gt_zero.T, self.shape), 1, mode='raise')
        if idx_zero.shape[0] > self.n_news:
            insert_idx = idx_zero[np.argsort(np.random.uniform(size=idx_zero.shape[0]))[:self.n_news], :]
        elif idx_zero.shape[0] > 0 and idx_zero.shape[0] <= self.n_news:
            insert_idx = idx_zero
        else:
            self.last_move = True
            return
        np.put(self.board, np.ravel_multi_index(insert_idx.T, self.shape), 2, mode='raise')
    
    def next_move(self, action_n):
        score = self.score
        if self.last_move:
            if len(self.actions_taken) == len(self.actions):
                return None, score
            else:
                if action_n not in self.actions_taken:
                    self.actions_taken.append(action_n)
        action = self.actions[action_n]
        self.next_board = np.copy(self.board)
        if action[1]:
            board = np.flip(self.next_board)
        else:
            board = self.next_board
        if action[0]:
            board = board.T
        for x in board:
            for i in range(n-1):
                if x[i+1] == 0:
                    x[i], x[i+1] = x[i+1], x[i]
                elif x[i] == x[i+1]:
                    x[i+1] *= 2
                    x[i] = 0
                    score += x[i+1]
                    if self.last_move:
                        self.last_move = False
                        self.actions_taken = []
        return self.next_board, score
    
    def update_move(self, board, score):
        self.board = board
        self.score = score
        self.add_numbers()

In [0]:
n = 4
game = Game2048(n=n)

In [4]:
game.reset()
while 1:
    action_n = int(np.random.uniform()*len(game.actions))
    new_board, score =  game.next_move(action_n=action_n)
    if new_board is None:
        break
    game.update_move(new_board, score)
print(game.board, score)

[[ 8. 16.  4.  8.]
 [ 2.  4.  2.  4.]
 [ 4.  2.  8. 16.]
 [16.  8.  2.  8.]] 240.0
